In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.functions import window
from pyspark.sql.functions import desc, asc

spark = SparkSession \
    .builder \
    .appName("ContadorDePalavras") \
    .getOrCreate()

24/06/22 18:19:00 WARN Utils: Your hostname, MacBook-Air-de-Daniel-2.local resolves to a loopback address: 127.0.0.1; using 192.168.0.4 instead (on interface en0)
24/06/22 18:19:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/22 18:19:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.types import StructType

userSchema = StructType().add("timestamp", "timestamp").add("word", "string")

In [3]:
file_dir = spark \
    .readStream \
    .format("csv") \
    .schema(userSchema) \
    .option ("includeTimestamp", "true") \
    .option("header", "true") \
    .option("delimiter", ";") \
    .option("maxFilesPerTrigger", 1) \
    .load("./*.csv")
    

In [4]:
file_dir.printSchema

<bound method DataFrame.printSchema of DataFrame[timestamp: timestamp, word: string]>

In [5]:
words = file_dir.select(
    explode(
        split(file_dir.word, " ")
    ).alias("word"), file_dir.timestamp
)

In [6]:
words.isStreaming

True

In [7]:
windowedCounts = words.groupBy(
    window(words.timestamp, "10 minutes", "5 minutes"),
    words.word
    ).count()

In [12]:
query = windowedCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()
    
#query.awaitTermination()

24/06/22 18:20:56 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/k9/z6_7b2f55tldshd60cfk17180000gn/T/temporary-6236c886-d6a3-4b8a-812f-d327fcdd3b3b. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/06/22 18:20:56 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


24/06/22 18:20:56 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 1, schema size: 2
CSV file: file:///Users/dcassis/Documents/Pós_Graduação%20-%20Data%20Science%20and%20Learning%20Machine/Módulo%203%20-%20Técnicas%20para%20o%20processamento%20do%20Big%20Data/aula2_3_spark_streaming/fake_data_4.csv
24/06/22 18:20:56 ERROR Executor: Exception in task 0.0 in stage 10.0 (TID 205)
org.apache.spark.SparkUpgradeException: [INCONSISTENT_BEHAVIOR_CROSS_VERSION.PARSE_DATETIME_BY_NEW_PARSER] You may get a different result due to the upgrading to Spark >= 3.0:
Fail to parse '2023-05-17 23:17:29,elderberry' in the new parser. You can set "spark.sql.legacy.timeParserPolicy" to "LEGACY" to restore the behavior before Spark 3.0, or set to "CORRECTED" and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.ExecutionErrors.failToParseDateTimeInNewParserError(ExecutionErrors.scala:54)
	at org.apache.spark.sql.e